<center><h1>Geng_Haoxiang_HW7</h1></center>

Name: Haoxiang Geng
<br>
Github Username: Haoxiang310
<br>
USC ID: 8045015278

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    hamming_loss, classification_report, confusion_matrix, f1_score,
    silhouette_samples, silhouette_score, accuracy_score, calinski_harabasz_score
)
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.pipeline import Pipeline as ImblearnPipeline, make_pipeline as make_pipeline_imblearn
from imblearn.over_sampling import SMOTE
warnings.filterwarnings("ignore")

### (a) Download the Anuran Calls (MFCCs) Data Set

In [2]:
mfcc_data_path = "../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv"
mfcc_data = pd.read_csv(mfcc_data_path)
print(mfcc_data)

      MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
0          1.0  0.152936 -0.105586  0.200722  0.317201  0.260764  0.100945   
1          1.0  0.171534 -0.098975  0.268425  0.338672  0.268353  0.060835   
2          1.0  0.152317 -0.082973  0.287128  0.276014  0.189867  0.008714   
3          1.0  0.224392  0.118985  0.329432  0.372088  0.361005  0.015501   
4          1.0  0.087817 -0.068345  0.306967  0.330923  0.249144  0.006884   
...        ...       ...       ...       ...       ...       ...       ...   
7190       1.0 -0.554504 -0.337717  0.035533  0.034511  0.443451  0.093889   
7191       1.0 -0.517273 -0.370574  0.030673  0.068097  0.402890  0.096628   
7192       1.0 -0.582557 -0.343237  0.029468  0.064179  0.385596  0.114905   
7193       1.0 -0.519497 -0.307553 -0.004922  0.072865  0.377131  0.086866   
7194       1.0 -0.508833 -0.324106  0.062068  0.078211  0.397188  0.094596   

      MFCCs_ 8  MFCCs_ 9  MFCCs_10  ...  MFCCs_17  MFCCs_18  MF

In [3]:
df = pd.DataFrame(mfcc_data)
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

train_x = train_df.iloc[:,:-4]
train_family = train_df["Family"]
train_genus = train_df["Genus"]
train_species = train_df["Species"]

test_x = test_df.iloc[:,:-4];
test_y = test_df.iloc[:,-4:-1]
test_family = test_df["Family"]
test_genus = test_df["Genus"]
test_specie = test_df["Species"]

### (b) Train a classifier for each label

#### (i) Research

- Exact Match Ratio    
The Exact Match Ratio is the strictest metric for evaluating multi-label classification, as it requires the predicted set of labels to exactly match the true set of labels for a sample to be considered correct.

- Hamming Loss    
The Hamming Loss is a more forgiving metric than the Exact Match Ratio. It measures the fraction of the wrong labels to the total number of labels. In other words, it computes the Hamming distance between the true and predicted labels.

#### (ii) Train a SVM for each of the labels

In [4]:
def find_parameter_bounds(X_train, y_train, threshold=0.7):
    C_values = np.logspace(-3, 6, 10)
    gamma_values = np.linspace(0.1, 2, 20)
    bounds = []

    for C in C_values:
        for gamma in gamma_values:
            svm = SVC(kernel='rbf', C=C, gamma=gamma)
            svm.fit(X_train, y_train)
            train_accuracy = accuracy_score(y_train, svm.predict(X_train))
            if train_accuracy >= threshold:
                bounds.append((C, gamma, train_accuracy))

    return bounds

#confirming the bound of crossvalidation
# print(find_parameter_bounds(train_x,train_family));

In [5]:
def train_evaluate_svm(X_train, y_train, X_test, y_test, C_range, gamma_range):

    param_grid = dict(gamma=gamma_range, C=C_range)
    svm = SVC(kernel='rbf')
    grid = GridSearchCV(svm, param_grid=param_grid, cv=10, scoring='accuracy')
    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    best_score = grid.best_score_

    # Train the SVM with the best parameters found
    best_svm = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'])
    best_svm.fit(X_train, y_train)

    # Predict on the test set
    y_pred = best_svm.predict(X_test)

    # Calculate test set accuracy
    test_accuracy = accuracy_score(y_test, y_pred)

    return best_svm, {
        'best_params': best_params,
        'best_score': best_score,
        'test_accuracy': test_accuracy
    }

trained_svms = []
results = {}
C_values = np.logspace(-3, 6, 10)
gamma_values = np.linspace(0.1, 2, 20)
best_params = {}

for label in ['Family', 'Genus', 'Species']:
    svm_model, result = train_evaluate_svm(
        train_x,
        train_df[label],
        test_x,
        test_df[label],
        C_values,
        gamma_values
    )
    trained_svms.append(svm_model)
    results[label] = result
    best_params[label] = result['best_params']

for label, metrics in results.items():
    print(f"Label: {label}")
    print(f"Best Parameters: {metrics['best_params']}")
    print(f"Best CV Accuracy: {metrics['best_score']}")
    print(f"Test Set Accuracy: {metrics['test_accuracy']}\n")

Label: Family
Best Parameters: {'C': 10.0, 'gamma': 2.0}
Best CV Accuracy: 0.9934480419072864
Test Set Accuracy: 0.9939786938397406

Label: Genus
Best Parameters: {'C': 100.0, 'gamma': 1.5}
Best CV Accuracy: 0.9908647322414719
Test Set Accuracy: 0.9907364520611394

Label: Species
Best Parameters: {'C': 10.0, 'gamma': 1.8}
Best CV Accuracy: 0.9904675123860013
Test Set Accuracy: 0.9902732746641963



In [6]:
def multi_label_prediction(test_x, trained_svms, test_y):
    predictions = []
    for svm in trained_svms:
        predictions.append(svm.predict(test_x))
    test_predictions = np.array(predictions).T
#     print(test_predictions)
    
    #exact match
    exact_match_count = np.all(test_predictions == test_y, axis=1).sum()
    exact_match = exact_match_count / len(test_y)
    
    #converting to binary label for calling hamming_loss
    mlb = MultiLabelBinarizer()
    test_y_transformed = mlb.fit_transform(test_y)
    test_predictions_transformed = mlb.transform(test_predictions)
    
    #hammingloss
    hamming_loss_value = hamming_loss(test_y_transformed, test_predictions_transformed)

    return exact_match, hamming_loss_value

test_y = np.array(test_y)
exact_match, hamming_loss_value = multi_label_prediction(test_x, trained_svms, test_y)
print(f"Exact Match: {exact_match}")
print(f"Hamming Loss: {hamming_loss_value}")

Exact Match: 0.9874942102825383
Hamming Loss: 0.002273779948629416


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [7]:
def train_evaluate_svm_l1(X_train, y_train, X_test, y_test, C_range):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', LinearSVC(penalty='l1', dual=False, max_iter=1000))
    ])
    param_grid = {'svm__C': C_range}
    grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)
    coefficients = best_model.named_steps['svm'].coef_
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return best_model, {
        'best_params': best_params['svm__C'],
        'best_score': best_score,
        'test_accuracy': test_accuracy,
        'coefficients': coefficients
    }

results = {}
C_values = np.logspace(-3, 6, 10)
best_params = {}
trained_svms_l1 = []

for label in ['Family', 'Genus', 'Species']:
    best_model, result = train_evaluate_svm_l1(
        train_x, 
        train_df[label], 
        test_x, 
        test_df[label], 
        C_values
    )
    trained_svms_l1.append(best_model) 
    results[label] = result

# print(results)

for label, metrics in results.items():
    print(f"Label: {label}")
    print(f"Best Parameters: C = {metrics['best_params']}")
    print(f"Best CV Accuracy: {metrics['best_score']}")
    print(f"Test Set Accuracy: {metrics['test_accuracy']}")
#     print(f"Coefficients (Weights): {metrics['coefficients']}\n")
    
exact_match, hamming_loss_value = multi_label_prediction(test_x, trained_svms_l1, test_y)
print(f"Exact Match: {exact_match}")
print(f"Hamming Loss: {hamming_loss_value}")

Label: Family
Best Parameters: C = 1.0
Best CV Accuracy: 0.9406288459717882
Test Set Accuracy: 0.9286706808707735
Label: Genus
Best Parameters: C = 10.0
Best CV Accuracy: 0.9527422764997319
Test Set Accuracy: 0.9416396479851783
Label: Species
Best Parameters: C = 10.0
Best CV Accuracy: 0.9602847202499291
Test Set Accuracy: 0.9592403890690134
Exact Match: 0.9129226493747105
Hamming Loss: 0.015495389279548612


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [8]:
def train_evaluate_svm_l1_with_smote(X_train, y_train, X_test, y_test, C_range):
    pipeline = make_pipeline_imblearn(
        StandardScaler(),
        SMOTE(),
        LinearSVC(penalty='l1', dual=False, max_iter=1000)
    )

    param_grid = {'linearsvc__C': C_range}
    grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)
    coefficients = best_model.named_steps['linearsvc'].coef_
    y_pred = best_model.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)

    return best_model, {
        'best_model': best_model, 
        'best_params': best_params['linearsvc__C'],
        'best_score': best_score,
        'test_accuracy': test_accuracy,
        'coefficients': coefficients
    }

results = {}
C_values = np.logspace(-3, 6, 10)
best_params = {}
trained_svms_l1_smote = []

for label in ['Family', 'Genus', 'Species']:
    best_model, result = train_evaluate_svm_l1_with_smote(
        train_x, 
        train_df[label], 
        test_x, 
        test_df[label], 
        C_values
    )
    trained_svms_l1_smote.append(result['best_model']) 
    results[label] = result

for label, metrics in results.items():
    print(f"Label: {label}")
    print(f"Best Parameters: C = {metrics['best_params']}")
    print(f"Best CV Accuracy: {metrics['best_score']}")
    print(f"Test Set Accuracy: {metrics['test_accuracy']}")
#     print(f"Coefficients (Weights): {metrics['coefficients']}\n")

exact_match, hamming_loss_value = multi_label_prediction(test_x, trained_svms_l1_smote, test_y)
print(f"Exact Match: {exact_match}")
print(f"Hamming Loss: {hamming_loss_value}")

Label: Family
Best Parameters: C = 1000000.0
Best CV Accuracy: 0.9227590804380069
Test Set Accuracy: 0.9101435849930524
Label: Genus
Best Parameters: C = 10000.0
Best CV Accuracy: 0.9138241976711161
Test Set Accuracy: 0.9031959240389069
Label: Species
Best Parameters: C = 100000.0
Best CV Accuracy: 0.9563152829057401
Test Set Accuracy: 0.9606299212598425
Exact Match: 0.8554886521537749
Hamming Loss: 0.020548233609836205


### Conclusion: SMOTE increase the hamming loss and decrease the exact match ratio.

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering

In [24]:
def monteCarlo(times, X, Y):
    hamming_distances = []
    hamming_losses = []
    hamming_scores = []

    for i in range(times):
        optimalK = getOptimalK(X, 50)
        clusterer = KMeans(n_clusters=optimalK, random_state=i)
        cluster_labels = clusterer.fit_predict(X)
        cluster_major = getMajority(optimalK, cluster_labels, Y)
        hamming_dist, hamming_loss_ = calculateHammingMetrics(cluster_major, cluster_labels, Y)
        hamming_score_ = 1 - hamming_loss_
        hamming_distances.append(hamming_dist)
        hamming_losses.append(hamming_loss_)
        hamming_scores.append(hamming_score_)
#         print(type(hamming_dist))
#         print(type(hamming_loss_))
#         print(type(hamming_score_))
        print(f"Iteration {i+1}: Hamming Distance = {hamming_dist:.4f}, Hamming Loss = {hamming_loss_:.4f}, Hamming Score = {hamming_score_:.4f}")
    
    avg_hamming_distance = np.mean(hamming_distances)
    std_hamming_distance = np.std(hamming_distances)
    
    avg_hamming_loss = np.mean(hamming_losses)
    std_hamming_loss = np.std(hamming_losses)

    avg_hamming_score = np.mean(hamming_scores)
    std_hamming_score = np.std(hamming_scores)

    print("\nSummary of Hamming Metrics:")
    print(f"Average Hamming Distance: {avg_hamming_distance}, Standard Deviation: {std_hamming_distance}")
    print(f"Average Hamming Loss: {avg_hamming_loss}, Standard Deviation: {std_hamming_loss}")
    print(f"Average Hamming Score: {avg_hamming_score}, Standard Deviation: {std_hamming_score}")

    return avg_hamming_distance, std_hamming_distance, avg_hamming_loss, std_hamming_loss, avg_hamming_score, std_hamming_score

In [10]:
#use ch score to determine the optimal k
def getOptimalK(X, max_k):
    optimalK, max_score = 2, 0
    for n in range(2, max_k + 1):
        clusterer = KMeans(n_clusters=n, random_state=42)
        cluster_labels = clusterer.fit_predict(X)
        ch_score = calinski_harabasz_score(X, cluster_labels)
        if ch_score > max_score:
            optimalK = n
            max_score = ch_score
    print(f"\nThe optimal K is: {optimalK}")
    return optimalK

### (b) Determine which family is the majority

In [11]:
def getMajority(numberOfClusters, clusterLabels, trueLabels):
    mostCommonLabels = []
    for clusterIndex in range(numberOfClusters):
        sampleIndicesInCluster = np.where(clusterLabels == clusterIndex)[0]
        labelsInCluster = trueLabels.iloc[sampleIndicesInCluster]
        mostCommonLabel = labelsInCluster.mode().iloc[0]
        mostCommonLabels.append(mostCommonLabel)
        print(f"Cluster {clusterIndex}: Most Common Label = {mostCommonLabel}")       
    return pd.DataFrame(mostCommonLabels)

### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

In [27]:
#calculate hammingloss
def customHammingLoss(predicted_labels, true_labels):
    predicted_df = pd.DataFrame(predicted_labels, columns=true_labels.columns)
    mismatches = np.sum(np.not_equal(predicted_df, true_labels).values)
    total_labels = np.prod(true_labels.shape)
    hamming_loss = mismatches / total_labels
    return hamming_loss

def calculateHammingMetrics(mostCommonLabels, clusterLabels, trueLabels):
    totalHammingDistance = 0
    predicted_labels = []

    for clusterIndex in range(len(mostCommonLabels)):
        sampleIndices = np.where(clusterLabels == clusterIndex)[0]
        clusterTrueLabels = trueLabels.iloc[sampleIndices]
        hammingDistance = np.sum(np.not_equal(clusterTrueLabels, mostCommonLabels.iloc[clusterIndex]), axis=1)
        totalHammingDistance += hammingDistance.sum()
        for index in sampleIndices:
            predicted_labels.append(mostCommonLabels.iloc[clusterIndex])

    predicted_labels_df = pd.DataFrame(predicted_labels, columns=trueLabels.columns)

    averageHammingDistance = totalHammingDistance / (len(clusterLabels) * len(trueLabels.columns))
    overallHammingLoss = customHammingLoss(predicted_labels_df, trueLabels)
    
    print(type(overallHammingLoss))
    return averageHammingDistance, overallHammingLoss

In [28]:
monteCarlo(50, mfcc_data.iloc[:, :-4], mfcc_data.iloc[:, -4:-1])


The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
Cluster 1: Most Common Label = Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
<class 'numpy.float64'>
Iteration 1: Hamming Distance = 0.2985, Hamming Loss = 0.3804, Hamming Score = 0.6196

The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
Cluster 1: Most Common Label = Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
<class 'numpy.float64'>
Iteration 2: Hamming Distance = 0.2985, Hamming Loss = 0.3804, Hamming Score = 0.6196

The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHyla


The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
Cluster 1: Most Common Label = Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
<class 'numpy.float64'>
Iteration 21: Hamming Distance = 0.2985, Hamming Loss = 0.3804, Hamming Score = 0.6196

The optimal K is: 2
Cluster 0: Most Common Label = Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
Cluster 1: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
<class 'numpy.float64'>
Iteration 22: Hamming Distance = 0.2985, Hamming Loss = 0.8353, Hamming Score = 0.1647

The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHy

<class 'numpy.float64'>
Iteration 40: Hamming Distance = 0.2985, Hamming Loss = 0.3804, Hamming Score = 0.6196

The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
Cluster 1: Most Common Label = Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
<class 'numpy.float64'>
Iteration 41: Hamming Distance = 0.2985, Hamming Loss = 0.3804, Hamming Score = 0.6196

The optimal K is: 2
Cluster 0: Most Common Label = Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
Cluster 1: Most Common Label = Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
<class 'numpy.float64'>
Iteration 42: Hamming Distance = 0.2985, Hamming Loss = 0.3804, Hamming Score = 0.6196

The optimal K is: 2
Cluster 0:

(0.29854065323141077,
 5.551115123125783e-17,
 0.5896316886726894,
 0.22674370071954272,
 0.4103683113273107,
 0.22674370071954278)

## 3. ISLR 12.6.2

# !!! If you cannot see the image in jupyter notebook, please refer to ../resources/ISLR12.6.2.jpg. I put the image in the resources folder.

![ISLR 12.6.2](../resources/ISLR12.6.2.jpg)